In [29]:
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.5/311.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 6.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.2 MB/s eta 0:00:0000:0100:01m


In [19]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.23.3
    Uninstalling openai-1.23.3:
      Successfully uninstalled openai-1.23.3
Note: you may need to restart the kernel to use updated packages.


In [3]:
import openai
import os
import re
import ast

openai.api_key = "sk-proj-g26sLhK5QyKW1Opp7v3RT3BlbkFJBtOAZ0MpwIGRQaxAFSJX"

In [16]:
from tqdm import tqdm


def format_prompt(utterances, speakers):
    prompt = ""
    for utterance, speaker in zip(utterances, speakers):
        prompt += f"{speaker}: \"{utterance}\"\n"
    return prompt

def predict_emotions(utterances, speakers):
    prompt = format_prompt(utterances, speakers)
    print(prompt)
    
    SYSTEM_PROMPT = "You are a smart and intelligent Emotion Cause Pair Extraction system. I will provide you the dictionary that contains one of the key as Utterance ID and value as a list of utterances and another key as the speaker and its value as a list of speaker that are a part of the conversation and you need to predict the emotions for each of the utternance keeping in mind the previous utterance and the output in given format."

    USER_PROMPT_1 = "Are you clear about your role?"

    ASSISTANT_PROMPT_1 = "Sure, I'm ready to help you with your NER task. Please provide me with the necessary information to get started."
    
    PROMPT = (
        "Emotions Definition:\n"
        "1. Anger: A strong feeling of annoyance, displeasure, or hostility.\n"
        "2. Joy: A feeling of great pleasure and happiness.\n"
        "3. Fear: An unpleasant emotion caused by the threat of danger, pain, or harm.\n"
        "4. Disgust: A feeling of revulsion or strong disapproval aroused by something unpleasant or offensive.\n"
        "5. Neutral: Not supporting or helping either side in a conflict, disagreement, etc.; impartial.\n"
        "6. Surprise: A feeling of mild astonishment or shock caused by something unexpected.\n"
        "7. Sadness: A feeling of deep distress caused by loss, disappointment, or other misfortune suffered by oneself or others.\n"
        "\n"
        "Input Format:\n"
        "A new line separated data in the form - Speaker:utterance\n"
        "Output Format:\n"
        "{{'Utterance': 'utterance text', 'Emotion': 'emotion'}}\n"
        "If no emotion is presented in any categories keep it Neutral\n"
    )
    
    final_prompt = PROMPT.format(prompt)
    
    response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": USER_PROMPT_1},
                    {"role": "assistant", "content": ASSISTANT_PROMPT_1},
                    {"role": "user", "content": final_prompt}
                ]
            )

    return response['choices'][0]['message']['content'].strip(" \n")

import time
def make_request_with_retry(utterances, speakers, retries=5, delay=60):
    while retries > 0:
        try:
            return predict_emotions(utterances, speakers)
        except openai.error.RateLimitError as e:
            print("Rate limit exceeded, retrying after delay...")
            time.sleep(delay)  # Wait for the specified delay
            retries -= 1
    raise Exception("API request failed after retries")


import json

def load_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data

def process_conversations(data):
    
    results = []
    for conversation in tqdm(data):
        utterances = [utterance['text'] for utterance in conversation['conversation']]
        speakers = [utterance['speaker'] for utterance in conversation['conversation']]
        predicted_emotions = predict_emotions(utterances, speakers)
        print(predict_emotions)
        # predicted_emotions = make_request_with_retry(utterances, speakers)
        results.append(predicted_emotions)
    return results

# Example usage:
train_data = load_data('/Users/parthivdholaria/Desktop/PARTHIV/IIIT DELHI/SEM6/NLP/NLP-Project/Dataset/Original_Dataset/Subtask_1_test.json')
results = process_conversations(train_data)

# Print results or process further as needed
print(results)

  0%|          | 0/1374 [00:00<?, ?it/s]

Chandler: "Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked ."
All: "Oh , yeah . Had that dream ."
Chandler: "Then I look down , and I realize there is a phone ... there ."
Joey: "Instead of ... ?"
Chandler: "That is right ."
Joey: "Never had that dream ."
Phoebe: "No ."
Chandler: "All of a sudden , the phone starts to ring ."



  0%|          | 0/1374 [00:00<?, ?it/s]


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.